In [ ]:
#!pip install numpy
#!pip install pandas
# #!pip install matplotlib
# !pip install transformers
# !pip install finbert-embedding transformers
# !pip install swifter

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib as plt
import swifter
# Imports for reading input directory
import os
#import torch
#import shutil
#the encoding detection and handling logic using the chardet
import chardet

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

In [ ]:
print("bert")

In [ ]:
##########################################################################
# numlabels is set to 2 as Bert can classify using 3 sentiments:         #
# positive, negative and neutral                                         #
##########################################################################
model_name = "ProsusAI/finbert"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
##########################################################################
# Preprocess function : This piece of code forms a embedded form for     #
# the input text                                                         #
##########################################################################
def preprocess_text(text):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    return inputs

In [ ]:
##########################################################################
# This piece of code takes text as input and finds the                   #
# softmax probablitites (from logits) for positive and negative class    #
##########################################################################
def get_sentiment_scores(text):
    inputs = preprocess_text(text)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)
    return probabilities.tolist()[0]

In [ ]:
#def process_csv_files(folder_path):
    #Read the csv file from the folder
   # data = pd.read_csv(folder_path + 'cleaned_text.csv')
    #Apply get_sentiment_scores function for sentiment scores on the csv file text data (here column: 'Cleaned Text')
   # data[['Positive', 'Neutral', 'Negative']] = data['Cleaned Text'].swifter.apply(lambda x: pd.Series(get_sentiment_scores(x)))
    #Drop the text variable; Keep only the scores and id
    #data_sub = data.drop('Cleaned Text',axis =1)
    # return the smaller dataset
    #return data_sub

In [ ]:
##############################################################
# This fuction reads the input directory with parsed data    #
# It captures the accession number                           #
# And calculates the sentiment score of the text             #
# Finally creates a dataframe for same                       #
##############################################################
#preprocess_text_file system
def process_text_files(folder_path):
    #Define the results as empty array
    results = []
    count=1
    # Iterate over files in the input directory
    for filename in os.listdir(folder_path):
        #Find the filepath
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
        else:
            continue

        if os.path.isfile(file_path):  # Check if the path is a file
            with open(file_path, 'rb') as file:
                raw_data = file.read()
                encoding = chardet.detect(raw_data)['encoding']

            try:
                text = raw_data.decode(encoding)
            except UnicodeDecodeError:
                print(f"Error decoding file: {file_path} with encoding: {encoding}. Skipping...")
                continue
            #keeping file name as the identifier
            file_id = filename.split('.')[0]

            #calculate the sentiments for each text file
            print(count)
            scores = get_sentiment_scores(text)
            count = count+1
            #Store results as dictionary
            result ={'id':file_id,'Positive':scores[0],'Neutral':scores[1],'Negative':scores[2]}
            #Append the results to final results
            results.append(result)

    #Create dataframe from the results
    print('generating dataframe')
    df = pd.DataFrame(results)

    #return df
    return df




In [ ]:
def generate_csv(df,filename,download_path):
    csv_data = df.to_csv(index=False)
    csv_file = f"{filename}.csv"
    # Save the CSV data to a file
    with open(f'{download_path}/{csv_file}', 'w') as file:
        file.write(csv_data)
    #df.to_csv(csv_file,compression='gzip')
    print('CSV Generated')


In [ ]:
# testing generate_csv function using the process_text_files
folder_path= '' #Specify the folder path where all the text files are present
output_filename = "Folder1"
download_path = "" #Specify the folder path where you want to save the result
df_text = process_text_files(folder_path)
#iteration no var
generate_csv(df_text,output_filename,download_path)